In [1]:
import sys
sys.path.append("eval/")
sys.path.append("models/")

import glob
import itertools
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import pickle as pkl
import torch

from class_mve_loss import T_MVELoss

import util_eval
import util_gcnn
import util_plot
import util_results

from setup import *

%load_ext autoreload
%autoreload 2
plt.rcParams.update({'font.size': 16})

## Hyperparameters

In [2]:
out_folder = "2104_rc_new1/"

dist='tnorm'
criterion = T_MVELoss()
max_lookback = 6
difference = True
device = torch.device("cpu")

# 95% pct
z=1.28

In [3]:
result_in = "rail_catchment_mve_results.csv"
results = pd.read_csv(project_dir+"results/"+out_folder+result_in)
results=results[results['Adjacency'] == 'func_euc_con_net']
results=results[results['Lookback'] == 2]

In [7]:
result_out = "rail_catchment_mve_results_val_80.csv"
ro = pd.read_csv(project_dir+"results/"+out_folder+result_out)
existing = ro[['Period','Train Extent','Adjacency','Predict Horizon','Time Size','Lookback','Model','spatial']].to_numpy()

In [8]:
for (p,ex,adj_type,ph,ts,l,ii,si) in zip(results['Period'],results['Train Extent'],results['Adjacency'],
                                 results['Predict Horizon'], results['Time Size'], results['Lookback'], 
                                 results['Model'], results['spatial']):
    if (p,ex,adj_type,ph,ts,l,ii,si) in existing:
        print('skip')
        continue
        
    n_time = 96//ts - 7

    # Get Data
    data, adj, spatial, downtown_filter = \
        util_gcnn.combine_datasources(project_dir, p, ph, ts, difference, max_lookback, l)

    (num_train, _, _, n_modes) = data['x'][0].shape
    (num_val, _, _, _) = data['x'][1].shape
    (num_test, _, _, _) = data['x'][2].shape

    n_stations = np.sum(downtown_filter)
    
    # Load Trained NN
    net = util_gcnn.load_model(project_dir, out_folder, p, ex, adj_type, ph, ts, l, ii, n_modes, n_stations, n_time)
    
    if net is None:
        continue
    
    # Turn Data into Torch Tensors
    _, _, valloader, testloader, adj_torch, spatial_torch, y_train_eval, y_val_eval, y_test_eval = \
                util_gcnn.prepare_for_torch(device, ex, data, adj, spatial, 
                                            downtown_filter, adj_type.split('_'))
    if si == 0:
        spatial_torch=None
    
    # Test Set    
    test_out_loc, test_out_std, test_loss = util_gcnn.testset_output_gcn(testloader, False, net, criterion, adj_torch, 
                                    spatial_torch, device, n_time, return_components=False)
    if dist == "lognorm":
        test_out_loc = np.exp(test_out_loc - np.power(test_out_std,2))
    mae, mse, _, _, _ = util_eval.eval_mean(test_out_loc, y_test_eval, 'Test')
    u, ub, uv, uc = util_eval.eval_theils(test_out_loc, y_test_eval)
    mpiw, picp = util_eval.eval_pi(test_out_loc - z*test_out_std, test_out_loc + z*test_out_std, y_test_eval)

    # Validation Set
    val_out_loc, val_out_std, val_loss = util_gcnn.testset_output_gcn(valloader, False, net, criterion, 
            adj_torch, spatial_torch, device, n_time)
    if dist == "lognorm":
        val_out_loc = np.exp(val_out_loc - np.power(val_out_std,2))
    val_mae, val_mse, _, _, _ = util_eval.eval_mean(val_out_loc, y_val_eval, 'val')
    val_u, val_ub, val_uv, val_uc = util_eval.eval_theils(val_out_loc, y_val_eval)
    val_mpiw, val_picp = util_eval.eval_pi(val_out_loc - z*val_out_std, val_out_loc + z*val_out_std, y_val_eval)
    
    with open(project_dir+"results/"+out_folder+result_out,"a") as f:
        f.write("%s,%s,%s,%s,%s,%d,%d,%d,%d,%d,%.2f,%.2f,%.2f,%.6f,%.6f,%.2f,%.2f,%.2f,%.6f,%.6f,%.6f,%.6f,%.6f,%.6f,%.6f,%.6f,%.6f,%.6f\n" % 
                ("GCN", p, ex, ex, adj_type, ii, ph, ts, l, si,
                    val_loss/num_val, val_mae, val_mse, val_mpiw, val_picp,
                    test_loss/num_test, mae, mse, mpiw, picp,
                    val_u, val_ub, val_uv, val_uc, u, ub, uv, uc))
        